# Tropical Cyclons Viewer

This notebook generates an animation of a preselected tropical cyclone in the region of Upolu and Tongatapu Islands.

Table of contents:

* [Select tropical cyclone](#1)
* [Location map](#2)
* [Data visualization](#3)

In [ ]:
import sys
import os
import os.path as op

# basic
import matplotlib
from cartopy.io.img_tiles import Stamen
import cartopy.crs as ccrs
from IPython.display import HTML, display
import ipywidgets as widgets
from ipywidgets import interactive

sys.path.insert(0, op.join(os.getcwd(), '..', '..'))

# dependencies
from lib.storm_viewer import *
from lib.config import *

In [ ]:
import numpy as np
from matplotlib import pyplot as plt, animation

##  Select tropical cyclone 

In [ ]:
# load the cyclons to choose
path = op.join(os.getcwd(), '..', 'data', 'databases', 'TCs_1981_2020.nc')
tcs = load_cyclons(path)

# widgets...
name_widget = widgets.Dropdown(options = list(np.unique(tcs.name.values)),
                               description='Cyclone name:  ')
season_widget = widgets.Dropdown(description='Season:  ')

# update season_widget based on name_widget
def update(*args):
    tc_to_season = tcs.isel(storm=np.where(tcs.name.values==name_widget.value)[0])
    season_options = list(np.unique(tc_to_season.season.values).astype(int))
    season_widget.options = season_options
    
name_widget.observe(update)

# extract from tcs the choosed cyclone
def select_cyclon(name_widget, season_widget):
    tc = tcs.isel(storm=np.where(tcs.name.values==name_widget)[0])
    tc = tc.isel(storm=np.where(tc.season.values==season_widget)[0])
    
    return tc

# plot the interaction
tc = interactive(select_cyclon,
                 name_widget = name_widget,
                 season_widget = season_widget) 
display(tc)

## Previsualization of the TC track 

In [ ]:
cyclon = tc.result

In [ ]:
fig = plt.figure(figsize=(5, 5))
gs = fig.add_gridspec(1,1)
ax = fig.add_subplot(gs[0], projection=ccrs.PlateCarree(central_longitude=180))

ax.plot(cyclon.squeeze().lon.dropna(dim='date_time'), cyclon.squeeze().lat.dropna(dim='date_time'), c='lavender', transform=ccrs.PlateCarree(), zorder=1)
im2 = ax.scatter(cyclon.lon, cyclon.lat, c=cyclon.wmo_pres,  s=25, cmap='rainbow', transform=ccrs.PlateCarree(), zorder=2)
ax.coastlines(resolution='auto', color='k')
ax.add_feature(cfeature.LAND)
ax.add_feature(cfeature.OCEAN)

plt.colorbar(im2, ax=ax, label='Pressure', shrink=0.4)
plt.show()

## TC animation

In [ ]:
paths = op.join(path, '..')
tc, temp, pres, mjo, awt, precip = load_data(tc.result, paths)

In [ ]:
pidx, id1, id2 = np.intersect1d(tc.time.astype('datetime64[ns]').values, temp.time.astype('datetime64[ns]').values, return_indices=True)
tc = tc.isel(date_time =id1)
tem = temp.isel(time=id2)

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111, projection=ccrs.PlateCarree(central_longitude=180))

im = ax.scatter([], [], c=[], vmin=tc.wmo_pres.min(), vmax=tc.wmo_pres.max(), zorder=5, cmap='viridis')
plt.colorbar(im, shrink=0.4).set_label('Wmo Pressure (mbar)')
im2 = ax.scatter([], [], c=[], vmin=22, vmax=32, zorder=5, cmap='jet')
plt.colorbar(im2, shrink=0.4).set_label('Temperature (ºC)')

def animate(frame):
    
    ax.cla()
    ax.plot(tc.lon, tc.lat,  transform=ccrs.PlateCarree(), c='k', zorder=4)
    ax.coastlines(resolution='auto', color='k')
    ax.add_feature(cfeature.LAND)
    ax.add_feature(cfeature.OCEAN)

    im2 = ax.pcolormesh(temp.lon, temp.lat, temp.isel(time=frame).sst, vmin=22, vmax=32, alpha=0.5, cmap='jet', transform=ccrs.PlateCarree(), zorder=1)
    
    ax.scatter(tc.isel(date_time=slice(None, frame)).lon, 
               tc.isel(date_time=slice(None, frame)).lat, 
               c=tc.isel(date_time=slice(None, frame)).wmo_pres,  s=20, 
               vmin=tc.wmo_pres.min(), vmax=tc.wmo_pres.max(), transform=ccrs.PlateCarree(),  cmap='viridis', zorder=5)
    
    ax.contour(pres.longitude, pres.latitude, pres.SLP[frame,:,:], 30, colors='darkgrey', transform=ccrs.PlateCarree(), zorder=2)
    ax.set_title('{0}'.format(tc.time.values[frame]))
    
anim = animation.FuncAnimation(fig, animate, interval=500, frames=len(tc.date_time) - 1)
plt.close()


In [ ]:
matplotlib.rcParams['animation.embed_limit'] = 2**32
HTML(anim.to_jshtml())